<a href="https://colab.research.google.com/github/MerakchiHibaa/FINAL_FUZZ2025/blob/main/FUZZ_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FTH requirements :

In [2]:
!pip install scikit-fuzzy

import networkx as nx
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
from numba import prange
from typing import List
import skfuzzy as fuzz
from scipy import signal
import numpy as np
from functools import lru_cache
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.6 MB/s eta 0:00:00


In [3]:
# FUZZ 2025

!wget https://gist.githubusercontent.com/MerakchiHibaa/09142b59721546554c5550a5c85703d8/raw/b953af93d867f7d937c4e2a838696801caa584ae/Fuzz_activity.txt -q --show-progress

Fuzz_activity.txt   100%[===================>]     115  --.-KB/s    in 0s      


In [4]:
# FUZZ 2025
path_onto = "Fuzz_activity.txt"

ontology_emd = nx.read_adjlist(path_onto, create_using=nx.DiGraph) # creates a directed graph (DiGraph) representation of the ontology data.



In [5]:
@lru_cache(maxsize=100000)
def wu_palmer_fth(x: str, y: str, path: str, rootnode="All") -> float:
    ontologie = nx.read_adjlist(path, create_using=nx.DiGraph)
    return (2.0 * nx.shortest_path_length(ontologie, rootnode, nx.lowest_common_ancestor(ontologie, x, y))) / (
            nx.shortest_path_length(ontologie, rootnode, x) + nx.shortest_path_length(ontologie, rootnode, y))



# Define the similarity used
def sim_FTH(x: str, y: str) -> float:
    return wu_palmer_fth(x, y, path_onto)

In [6]:
from typing import List, TypeVar

T = TypeVar('T') # defines a type variable and allows the class to work with generic types.
# https://dev.to/decorator_factory/typevars-explained-hmo


class Temporal_seq:
    def __init__(self, acts: List[T], times: List[float]) -> None:
        self.acts = acts
        self.times = times
        # acts: A list of elements of type T. The type variable T allows the list to contain elements of any type.
        # times: A list of floating-point numbers representing the corresponding times for each activity.

In [7]:
class Edit_FTH:# To define the edit operation
    def __init__(self, x: str, delta: float, t_edit: float, S_i: Temporal_seq):
        self.x = x # the activity
        self.delta = delta # the duration
        self.t_edit = t_edit # the starting point
        self.S_i = S_i # the sequence


In [8]:
def to_discrete(S_i: Temporal_seq, unit_of_time: float) -> List[str]:
  return [S_i.acts[k] for k in range(len(S_i.acts)) for j in range(int(S_i.times[k] / unit_of_time))]

## Standard versions :

In [9]:
def cost_gamma_fth(e: Edit_FTH, interval_step = 1, time_window = 240) -> float:
  I = np.arange(0, np.sum(e.S_i.times)  , interval_step) # the duration divided by the interval step
  mu = fuzz.trapmf(I, [e.t_edit - time_window, e.t_edit, e.t_edit + e.delta, # the context function
                       e.t_edit + e.delta + time_window])
  sim_fun = [sim_fth_e(e, t) for t in I] # the similarity measure,t starts with 0
  sim_context = [mu[i] * sim_fun[i] for i in range(len(I))] # the similarity multiplied by the context function
  tab_gate = np.arange(0, e.delta, interval_step)
  gate = fuzz.trapmf(tab_gate, [0, 0, e.delta, e.delta])
  convo = signal.convolve(sim_context, gate, mode='same') / (e.delta / interval_step) # calculates the convolution weighted by the duration divided by the interval step
  return 1 - np.max(convo) # returns the cost (between 0 and 1)

def cost_delta_fth(e: Edit_FTH, interval_step = 1, time_window = 240) -> float:
  return e.delta * cost_gamma_fth(e, interval_step, time_window) # returns the weighted cost (cost multiplied by the duration of the new activity )

# The function that calculates the one-sided distance between two trajectories
def one_sided_dist_fth(S1: Temporal_seq, S2: Temporal_seq, f = cost_delta_fth, interval_step = 1, time_window = 240) -> float:
  sum = 0

  for i in prange(len(S1.acts)): # starts from 0
    e = Edit_FTH(S1.acts[i], S1.times[i], np.sum(S1.times[:i]), S2) # np.sum(S1.times[:i]) calculates the cumulative time spent on actions prior to the current action at index i (to get the beginning of the activity)
    sum += f(e, interval_step, time_window) # f is the used cost function (delta or gamma function)
  return sum # sum represents the sum of maximal costs necessary for all the edit operations

# Relative version (with precision )

def cost_gamma_fth_relative(e: Edit_FTH, interval_step = 0.001, time_window = 0.5) -> float:
  # I = np.arange(0, round(np.sum(e.S_i.times), precision - 1)  , interval_step) # the duration divided by the interval step
  I = np.arange(0, np.sum(e.S_i.times)  , interval_step)
  # print("inside cost_gamma_fth_relative, e.t_edit ", e.t_edit , 'e.S_i.times ', e.S_i.times)
  mu = fuzz.trapmf(I, [e.t_edit - time_window, e.t_edit, e.t_edit + e.delta, # the context function
                       e.t_edit + e.delta + time_window])
  sim_fun = [sim_fth_e(e, t) for t in I] # the similarity measure
  sim_context = [mu[i] * sim_fun[i] for i in range(len(I))] # the similarity multiplied by the context function
  tab_gate = np.arange(0, e.delta, interval_step)
  gate = fuzz.trapmf(tab_gate, [0, 0, e.delta, e.delta])
  convo = signal.convolve(sim_context, gate, mode='same') / (e.delta / interval_step) # calculates the convolution
  # weighted by the duration divided by the interval step
  return 1 - np.max(convo) # returns the cost (between 0 and 1)

def cost_delta_fth_relative(e: Edit_FTH, interval_step = 0.001, time_window = 0.5) -> float:
  return e.delta * cost_gamma_fth_relative(e, interval_step, time_window) # returns the weighted cost (cost multiplied by the duration of the new activity )

# The function that calculates the one-sided distance between two trajectories
def one_sided_dist_fth_relative(S1: Temporal_seq, S2: Temporal_seq, f = cost_delta_fth_relative, interval_step = 0.001, time_window = 0.5) -> float:
  sum = 0 # 1
  for i in prange(len(S1.acts)):
    e = Edit_FTH(S1.acts[i], S1.times[i], np.sum(S1.times[:i]), S2) # np.sum(S1.times[:i]) calculates the cumulative time spent on actions prior to the current action at index i (to get the beginning of the activity)
    # print('inside one_sided_dist_fth_relative S1.times[i] ' , S1.times[i] )
    sum += f(e, interval_step, time_window) # f is the used cost function (delta or gamma function)
  return sum # sum represents the sum of maximal costs necessary for all the edit operations

In [10]:
def dist_fth(S1: Temporal_seq, S2: Temporal_seq, f = cost_delta_fth, interval_step = 1, time_window = 1, agg = max)-> float:
  return agg(one_sided_dist_fth(S1, S2, f, interval_step, time_window), # returns the maximal one-sided cost to get from S1 to S2 or from S2 to S1
             one_sided_dist_fth(S2, S1, f, interval_step, time_window))


# relative version

def dist_fth_relative(S1: Temporal_seq, S2: Temporal_seq, f = cost_delta_fth_relative, interval_step = 1/ 10 **3, time_window =  0.01, agg = max)-> float:
  # print('inside dist_fth_relative S1 = ', S1.acts, S1.times , 'S2 =', S2.acts, S2.times)
  return agg(one_sided_dist_fth_relative(S1, S2, f, interval_step, time_window), # returns the maximal one-sided cost to get from S1 to S2 or from S2 to S1
             one_sided_dist_fth_relative(S2, S1, f, interval_step, time_window))

## Version with flexible H (time window) :

In [11]:
def sim_fth_e(e: Edit_FTH, t: float) -> float: #  for t in I (starts with 0)
    for i in prange(len(e.S_i.times)):# Iterate through the times in the sequence 'e.S_i'
    # every i is an index for a duration (starts with 0)
        if np.sum(e.S_i.times[:i]) <= t < np.sum(e.S_i.times[:i + 1]): # # Check if 't' falls within the time interval of the current activity in the sequence
            return sim_FTH(e.S_i.acts[i], e.x) # If 't' is within the interval, return the similarity between the current activity and 'e.x'
    return 0

In [12]:
def fuzzy_context_fth_h(e: Edit_FTH, time_window = 480) -> List[float]:
  return fuzz.trapmf(np.arange(0, np.sum(e.S_i.times), interval_step),
                     [e.t_edit - time_window, e.t_edit, e.t_edit + e.delta, e.t_edit + e.delta + time_window])



In [13]:
def cost_gamma_fth_h(e: Edit_FTH, time_window, interval_step = 1 ) -> float:
  I = np.arange(0, np.sum(e.S_i.times), interval_step)
  mu = fuzz.trapmf(I, [e.t_edit - time_window, e.t_edit, e.t_edit + e.delta,
                       e.t_edit + e.delta + time_window])
  sim_fun = [sim_fth_e(e, t) for t in I]
  sim_context = [mu[i] * sim_fun[i] for i in range(len(I))]
  tab_gate = np.arange(0, e.delta, interval_step)
  gate = fuzz.trapmf(tab_gate, [0, 0, e.delta, e.delta])
  convo = signal.convolve(sim_context, gate, mode='same') / (e.delta / interval_step)
  return 1 - np.max(convo)

# time_window = 4
def cost_delta_fth_h(e: Edit_FTH,  time_window, interval_step = 1) -> float:
  return e.delta * cost_gamma_fth_h(e, time_window, interval_step)

In [14]:
# time_window =  = 720
def one_sided_dist_fth_h(S1: Temporal_seq, S2: Temporal_seq, time_window, f = cost_delta_fth_h, interval_step = 1) -> float:
  sum = 0
  for i in prange(len(S1.acts)):
    e = Edit_FTH(S1.acts[i], S1.times[i], np.sum(S1.times[:i]), S2)
    sum += f(e, interval_step, time_window)
  return sum

def dist_fth_h(S1: Temporal_seq, S2: Temporal_seq, time_window, f = cost_delta_fth_h, interval_step = 2, agg = max)-> float:
  return agg(one_sided_dist_fth_h(S1, S2, time_window, f, interval_step),
             one_sided_dist_fth_h(S2, S1,time_window, f, interval_step))

## Tests :

# Pattern searching :

In [15]:
# To find all possible subsequences
def contiguous_substrings(s: Temporal_seq) -> List[Temporal_seq]:
    substrings = []
    n = len(s.acts)
    for i in range(n):  # O(n)
        for j in range(i, n):  # O(n)
            acts_substring = s.acts[i:j+1]
            times_substring = s.times[i:j+1]
            substring = Temporal_seq(acts_substring, times_substring)
            substrings.append(substring)

    return substrings
def sum_duration(sequence: Temporal_seq) -> float: # gives the sum of durations for a sequence
    return sum(sequence.times)

In [16]:
# FUZZ 2025
def decode_activities(acts: List[str]) -> List[str]:
    activity_mapping = {
        "1": "Bike",
        "2": "Walk",
        "3": "Bus",
        "4": "Work",
        "5": "Shopping",
        "6": "Read",
        "7": "Swim",

    }
    decoded_activities = [activity_mapping[act] for act in acts]
    return decoded_activities

s_h = Temporal_seq(['1', '4', '5'], [30, 5, 12])
decoded_activities = decode_activities(s_h.acts)
print(decoded_activities)

['Bike', 'Work', 'Shopping']


### Relative FTH

In [17]:
import copy
def relative_temporal_seq(T :Temporal_seq ) -> Temporal_seq:
  T_ = copy.deepcopy(T)
  T_max_seq = sum_duration(T_)
  relative_times = [time / T_max_seq * 100 for time in T_.times]
  return Temporal_seq(T_.acts, relative_times)

In [18]:
def rounded_relative_temporal_seq(T: Temporal_seq) -> Temporal_seq:
    T_ = copy.deepcopy(T)
    T_max_seq = sum(T_.times)
    # relative_times = [round(time / T_max_seq*100, precision) for time in T_.times]
    relative_times = [time / T_max_seq * 1000 for time in T_.times]

    integer_parts = [int(time) for time in relative_times] # Truncate to integer part
    # print('integer_parts' , integer_parts)
    remainders = [time - int(time) for time in relative_times] # Calculate remainders
    # print('remainders' , remainders)
    # Rank remainders in decreasing order
    ranked_remainders = sorted([(i,integer_parts[i], remainders[i]) for i in range(len(integer_parts))], key=lambda x: x[2], reverse=True)


    sum_integers = sum(integer_parts)
    # print('sum_integers  ' , sum_integers)
    index = 0

    # While the sum of integers is less than 1000, add one to the integer with the next biggest integer
    while sum_integers < 1000:
        # print('inside while, sum_integers = ' , sum_integers)
        # print('index ', index)
        index %= len(ranked_remainders) # Wrap around if we reach the end of the ranked remainders list
        i, integer_part, _ = ranked_remainders[index] # Get the index of the next biggest remainder
        integer_parts[i] += 1 # Add one to the integer part
        sum_integers += 1 # Increment the sum of remainders
        index += 1 # Move to the next remainder


    return Temporal_seq(T_.acts, integer_parts)



In [19]:
# FUZZ 2025
precision = 3

import copy
# Shows the TOP-K per sequence
# We eliminate the subsequences that are too long or too short

def FTH_relative_elimin_k_fuzz(subb, pattern , k, degree, f = cost_delta_fth, decoding_func = decode_activities):
  result = []
  patt = copy.deepcopy(pattern)
  len_patt = sum_duration(patt)
  len_patt_h = len_patt/60

  relative_pattern = rounded_relative_temporal_seq(patt) # precision -1
  # for subb in substrings:
  sub = copy.deepcopy(subb)
  # print(sub.acts, sub.times)

  len_sub = sum_duration(sub)
  # print('sum_duration(sub) ', sum_duration(sub))
  len_sub_h = len_sub/60

  bigger, smaller = (len_sub, len_patt) if len_sub >= len_patt else (len_patt, len_sub)
  gap = smaller / bigger * 100
  if gap >= (degree*100) :
  # temp_sim = fuzzy_exclusion_fth_h(len_sub_h, len_patt_h, support_tronc)
  # T = len_sub if len_sub <= len_patt else len_patt

  # if temp_sim > 0 :

    relative_sub = rounded_relative_temporal_seq(sub ) # precision -1
    # print('================= inside FTH_relative_elimin_k_fuzz =========')
    # print("relative_pattern ", relative_pattern.acts, relative_pattern.times)
    # print("relative_sub ", relative_sub.acts, relative_sub.times)
    # distance = dist_fth_relative(relative_pattern, relative_sub, f, 1/ 10 **precision, 0.01, max ) # with the default parameters
    distance = dist_fth(relative_pattern, relative_sub, f,1, 500, max )/1000 # with the default parameters
    # support = 200
    result.append(( subb, distance))

  if result:
    sorted_result_fth = sorted(result, key=lambda x: x[1])
    k_smallest_pairs = sorted_result_fth[:k]

    print("TOP K:")
    # for substring, similarity, temp_s in k_smallest_pairs:
    for substring, distance in k_smallest_pairs:
        # print('inside for')
        decoded_sub = decoding_func(substring.acts)
        print( "Distance:", distance, " - Sequence:", decoded_sub, substring.times)


  else:
    print("No valid subsequences found.")



In [20]:
# FUZZ 2025
precision = 2
def FTH_relative_k_fuzz(subb, pattern, k, f = cost_delta_fth_relative): # degree is a value introduced by the user that represents the acceptable error? or the acceptable gap between the subsequence and the pattern
  result = []
  patt = copy.deepcopy(pattern)
  #relative_pattern = relative_temporal_seq(patt)
  # relative_pattern = rounded_relative_temporal_seq(patt, precision-1)
  relative_pattern = rounded_relative_temporal_seq(patt)
  # print("relative_pattern " , relative_pattern.acts, relative_pattern.times)

  # for subb in substrings:
  sub = copy.deepcopy(subb)
  relative_sub = rounded_relative_temporal_seq(sub ) #  precision - 1
  # distance = dist_fth_relative(relative_pattern, relative_sub, f, 1/ 10 **precision, 0.01, max ) # with the default parameters
  # def dist_fth(S1: Temporal_seq, S2: Temporal_seq, f = cost_gamma_fth, interval_step = 5, time_window = 480, agg = max)-> float:
  distance = dist_fth(relative_pattern, relative_sub, f, 1, 500 , max )/1000
  result.append(( subb, distance))

  if result:
    # print('if result')
    sorted_result_fth = sorted(result, key=lambda x: x[1])
    k_smallest_pairs = sorted_result_fth[:k]

    print("TOP K:")
    # for substring, similarity, temp_s in k_smallest_pairs:
    for substring, distance in k_smallest_pairs:
        # print('inside for')
        decoded_sub = decode_activities(substring.acts)
        print( "Distance:", distance, " - Sequence:", decoded_sub, substring.times)


  else:
    print("No valid subsequences found.")


In [24]:
# FUZZ 2025  pour tester FTH relatif :

# 1 => Bike
# 2 => Walk
# 3 => Bus
# 4 => Work
# 5 => Shopping
# 6 => Read
# 7 => Swim

# Define the sequences
S = Temporal_seq(
    ['2', '4', '2', '5', '2'],  # Activities: Walk, Work, Walk, Shopping, Walk
    [60, 240, 60, 120, 60]      # Durations in minutes: 1h, 4h, 1h, 2h, 1h
)

S1 = Temporal_seq(
    ['2', '5', '2', '4', '2'],  # Activities: Walk, Shopping, Walk, Work, Walk
    [30, 90, 30, 240, 30]       # Durations in minutes: 30min, 1h30, 30min, 4h, 30min
)

S2 = Temporal_seq(
    ['3', '4', '3'],            # Activities: Bus, Work, Bus
    [30, 420, 30]               # Durations in minutes: 30min, 7h, 30min
)

S22 = Temporal_seq(
    ['1', '4', '1'],            # Activities: Bus, Work, Bus
    [30, 420, 30]               # Durations in minutes: 30min, 7h, 30min
)

# S3 = Temporal_seq(
#     ['2', '4', '2', '5'],       # Activities: Walk, Work, Walk, Shopping
#     [60, 120, 30, 60]           # Durations in minutes: 1h, 2h, 30min, 1h
# )
# S33 = Temporal_seq(
#     ['1', '4', '1'],       # Activities: Bike, Work, Bike
#     [60, 240, 30]           # Durations in minutes: 1h, 2h, 30min
# )

# S333 = Temporal_seq(
#     ['1', '4', '1'],       # Activities: Bike, Work, Bike
#     [60, 120, 60]           # Durations in minutes: 1h, 2h, 1h
# )
# S4 = Temporal_seq(
#     ['2', '4', '2', '5', '2'] ,  # Activities: Walk, Work, Walk, Shopping, Walk
#     [80, 260, 80, 160, 80]      # Durations in minutes: 1h20, 4h, 1h, 2h, 1h
# )
# S4 = Temporal_seq(
#     ['2', '4', '2', '5', '2'] ,  # Activities: Walk, Work, Walk, Shopping, Walk
#     [80, 260, 80, 160, 80]      # Durations : 1h, 4h40, 1h20, 2h40, 1h20
# )

S3 = Temporal_seq(
    ['1', '4', '1'] ,  # Activities: Bike, Work, Bike
    [30, 300, 30]      # Durations : 30min, 5h, 30min
)

S4 = Temporal_seq(
    ['2', '4', '2', '5', '2'] ,  # Activities: Walk, Work, Walk, Shopping, Walk
    [65, 245, 65, 145, 80]      # Durations : 1h05, 4h25, 1h05, 2h25, 1h20
)
# Example usage
print("Sequence S:", S.acts , S.times)
print("Sequence S1:", S1.acts , S1.times)
print("Sequence S2:", S2.acts , S2.times)
print("Sequence S3:", S3.acts , S3.times)
print("Sequence S4:", S4.acts , S4.times)


Sequence S: ['2', '4', '2', '5', '2'] [60, 240, 60, 120, 60]
Sequence S1: ['2', '5', '2', '4', '2'] [30, 90, 30, 240, 30]
Sequence S2: ['3', '4', '3'] [30, 420, 30]
Sequence S3: ['1', '4', '1'] [30, 300, 30]
Sequence S4: ['2', '4', '2', '5', '2'] [65, 245, 65, 145, 80]


In [25]:
# FUZZ 2025
seq_example = [S, S1, S2, S3, S4]

#### FTH relative ( avec élimination)

In [26]:
# # FUZZ 2025
# seq_example = [S, S1, S2, S3]

# # FUZZ 2025
# import copy
# degree_length = 0.7

# print('______________________________FTH_delta relative (avec élimination)__________________________________')

# decoded_main = decode_activities(S.acts)
# print( "Pattern :", decoded_main, S.times)
# print('__________________________________________________________________________________')

# for rel_seq in seq_example :
#   print('rel_seq ', rel_seq.acts, rel_seq.times)
#   # relative_subs = contiguous_substrings(rel_seq)
# ```python
# FUZZ 2025

# FUZZ 2025
import copy
degree_length = 0.7

print('______________________________FTH_delta relative (avec élimination)__________________________________')

decoded_main = decode_activities(S.acts)
print( "Pattern :", decoded_main, S.times)
print('__________________________________________________________________________________')

for rel_seq in seq_example :
  print('rel_seq ', rel_seq.acts, rel_seq.times)
  # relative_subs = contiguous_substrings(rel_seq)
  decoded_seqqq = decode_activities(rel_seq.acts)
  print( "Sequence:", decoded_seqqq, rel_seq.times)
  FTH_relative_elimin_k_fuzz(rel_seq, S , 100,degree_length, f = cost_delta_fth_relative, decoding_func = decode_activities)

  print('__________________________________________________________________________________')



______________________________FTH_delta relative (avec élimination)__________________________________
Pattern : ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
__________________________________________________________________________________
rel_seq  ['2', '4', '2', '5', '2'] [60, 240, 60, 120, 60]
Sequence: ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
TOP K:
Distance: 0.0  - Sequence: ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
__________________________________________________________________________________
rel_seq  ['2', '5', '2', '4', '2'] [30, 90, 30, 240, 30]
Sequence: ['Walk', 'Shopping', 'Walk', 'Work', 'Walk'] [30, 90, 30, 240, 30]
TOP K:
Distance: 0.4205980000000001  - Sequence: ['Walk', 'Shopping', 'Walk', 'Work', 'Walk'] [30, 90, 30, 240, 30]
__________________________________________________________________________________
rel_seq  ['3', '4', '3'] [30, 420, 30]
Sequence: ['Bus', 'Work', 'Bus'] [30, 420, 3

In [27]:
# # FUZZ 2025
# seq_example = [S, S1, S2, S3]

# # FUZZ 2025
# import copy
# degree_length = 0.7

# print('______________________________FTH_delta relative (avec élimination)__________________________________')

# decoded_main = decode_activities(S.acts)
# print( "Pattern :", decoded_main, S.times)
# print('__________________________________________________________________________________')

# for rel_seq in seq_example :
#   print('rel_seq ', rel_seq.acts, rel_seq.times)
#   # relative_subs = contiguous_substrings(rel_seq)
# ```python
# FUZZ 2025

# FUZZ 2025
import copy
degree_length = 0.7

print('______________________________FTH_delta relative (avec élimination)__________________________________')

decoded_main = decode_activities(S.acts)
print( "Pattern :", decoded_main, S.times)
print('__________________________________________________________________________________')

for rel_seq in seq_example :
  print('rel_seq ', rel_seq.acts, rel_seq.times)
  # relative_subs = contiguous_substrings(rel_seq)
  decoded_seqqq = decode_activities(rel_seq.acts)
  print( "Sequence:", decoded_seqqq, rel_seq.times)
  FTH_relative_k_fuzz(rel_seq, S , 100, f = cost_delta_fth_relative)

  print('__________________________________________________________________________________')



______________________________FTH_delta relative (avec élimination)__________________________________
Pattern : ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
__________________________________________________________________________________
rel_seq  ['2', '4', '2', '5', '2'] [60, 240, 60, 120, 60]
Sequence: ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
TOP K:
Distance: 0.0  - Sequence: ['Walk', 'Work', 'Walk', 'Shopping', 'Walk'] [60, 240, 60, 120, 60]
__________________________________________________________________________________
rel_seq  ['2', '5', '2', '4', '2'] [30, 90, 30, 240, 30]
Sequence: ['Walk', 'Shopping', 'Walk', 'Work', 'Walk'] [30, 90, 30, 240, 30]
TOP K:
Distance: 0.4205980000000001  - Sequence: ['Walk', 'Shopping', 'Walk', 'Work', 'Walk'] [30, 90, 30, 240, 30]
__________________________________________________________________________________
rel_seq  ['3', '4', '3'] [30, 420, 30]
Sequence: ['Bus', 'Work', 'Bus'] [30, 420, 3

In [ ]:
# FUZZ 2025
def similarity_fth(fth_distance, threshold):
    """
    Computes the similarity score based on FTH distance and threshold.

    Parameters:
        fth_distance (float): The FTH distance between two sequences.
        threshold (float): The threshold value for similarity.

    Returns:
        float: The similarity score.
    """
    if fth_distance <= threshold:
        return 1 / (1 + fth_distance)
    else:
        return 0

# Example
fth_dist1 = 705.66
fth_dist2 = 513.33

# Threshold for similarity
T = 1440

similarity1 = similarity_fth(fth_dist1, T)
print(f"Similarity1: {similarity1}")
print('__________________________________________________________________________________')
similarity2 = similarity_fth(fth_dist2, T)
print(f"Similarity2: {similarity2}")


Similarity1: 0.0014151076896951858
__________________________________________________________________________________
Similarity2: 0.0019442770205898934


In [ ]:
# FUZZ 2025

def linear_decay(d, T):
    """
    Linear decay function that transforms a distance into a similarity score.

    Parameters:
    d (float or numpy array): The distance value(s).
    T (float): The threshold value. Similarity is 0 for distances greater than T.

    Returns:
    float or numpy array: Similarity value(s) between 0 and 1.
    """
    return np.maximum(1 - d / T, 0)

# Example usage:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
Threashold = 1440
fth_dist1 = 705.66
fth_dist2 = 513.33
# Compute similarity
similarity1 = linear_decay(fth_dist1, Threashold)
similarity2 = linear_decay(fth_dist2, Threashold)

print(f"Similarity1: {similarity1}")
print(f"Similarity2: {similarity2}")

Similarity1: 0.5099583333333333
Similarity2: 0.6435208333333333


In [30]:
# FUZZ 2025
def membership_function(x, b, w):
    """
    Computes the membership value μ(x) based on parameters b and w.

    Parameters:
        x (float): The input value.
        b (float): The center of the range.
        w (float): The width of the range.

    Returns:
        float: The membership value μ(x).
    """
    if x <= b - w or x >= b + w:
        return 0
    elif b - w < x <= b:
        return (x - (b - w)) / w
    elif b < x < b + w:
        return ((b + w) - x) / w

# Example usage:
x1 = 7  # Input value
x2 = 8  # Input value
x3 = 5  # Input value
x4 = 10
# x33 = 4.5  # Input value
# x333 = 4  # Input value
# x4 = 11  # Input value
# x44 = 10  # Input value

b = 9.0  # Center of the range
w = 4.0  # Width of the range

mu1 = membership_function(x1, b, w)
mu2 = membership_function(x2, b, w)
mu3 = membership_function(x3, b, w)
# mu33 = membership_function(x33, b, w)
# mu333 = membership_function(x333, b, w)
mu4 = membership_function(x4, b, w)
# mu44 = membership_function(x44, b, w)
print(f"Membership value 1: {mu1}")
print(f"Membership value 2: {mu2}")
print(f"Membership value 3: {mu3}")
# print(f"Membership value 33: {mu33}")
# print(f"Membership value 333: {mu333}")
print(f"Membership value 4: {mu4}")
# print(f"Membership value 44: {mu44}")

Membership value 1: 0.5
Membership value 2: 0.75
Membership value 3: 0
Membership value 4: 0.75
